In [ ]:
from extract.scraper import Scraper

In [ ]:
scraper = Scraper(headless=False)
scraper.scrape('cumplo')

In [ ]:
import pymongo
from bson.json_util import dumps

host = credentials()['mongodb']['host']
user = credentials()['mongodb']['user']
password = credentials()['mongodb']['password']

class SaveProjects(object):
    def __init__(self):
        uri = "mongodb+srv://{}:{}@{}/test?retryWrites=true".format(user,password,host)
        client = pymongo.MongoClient(uri)
        self._db = client['investment']
    
    def save(self, projects):
        self._db.project.insert_many(projects)

In [ ]:
save_project = SaveProjects()
save_project.save([{"a":1},{"b":2}])

In [ ]:
save_project._db.project.delete_many([])


In [ ]:
save_project._db.project.count()